In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import tree
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
import random
import shutil
import os

path = '../NumericDataSet/adult.csv'
data = pd.read_csv(path)

# remove rows where occupation is unknown
data = data[data.occupation != '?']
raw_data = data[data.occupation != '?']


In [ ]:
data['workclass_num'] = data.workclass.map({'Private':0, 'State-gov':1, 'Federal-gov':2, 'Self-emp-not-inc':3, 'Self-emp-inc':4, 'Local-gov':5, 'Without-pay':6})
data['marital_num'] = data['marital.status'].map({'Widowed':0, 'Divorced':1, 'Separated':2, 'Never-married':3, 'Married-civ-spouse':4, 'Married-AF-spouse':4, 'Married-spouse-absent':5})
data['race_num'] = data.race.map({'White':0, 'Black':1, 'Asian-Pac-Islander':2, 'Amer-Indian-Eskimo':3, 'Other':4})
data['sex_num'] = np.where(data.sex == 'Female', 0, 1)
data['rel_num'] = data.relationship.map({'Not-in-family':0, 'Unmarried':0, 'Own-child':0, 'Other-relative':0, 'Husband':1, 'Wife':1})
data['over50K'] = np.where(data.income == '<=50K', 0, 1)



#columns = ['workclass', 'marital.status', 'race','sex','relationship','income']
#data.drop(columns, inplace=True, axis=1)

df1 = pd.DataFrame(data)
df1.to_csv("../NumericDataSet/clean_adult.csv")
print("Saved New File As ../Census/clean_adult.csv")
data.head()

Saved New File As ../Census/clean_adult.csv


,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,...,capital.loss,hours.per.week,native.country,income,workclass_num,marital_num,race_num,sex_num,rel_num,over50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,...,4356,18,United-States,<=50K,0,0,0,0,0,0
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,...,3900,40,United-States,<=50K,0,1,0,0,0,0
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,...,3900,40,United-States,<=50K,0,2,0,0,0,0
5,34,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,...,3770,45,United-States,<=50K,0,1,0,0,0,0
6,38,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,...,3770,40,United-States,<=50K,0,2,0,1,0,0


In [ ]:
def getNoiseData(dataSet,percent):
    #dataSet = np.delete(dataSet,(0), axis=0)
    size = dataSet.shape
    rows = size[0]
    #print(rows)
    cols = size[1]
   
    num = (int)(rows*(percent/100))
    end = np.amax(dataSet,axis=0)
    start = np.amin(dataSet,axis=0)
    #print( start )
    #print( end )
    
    #print(start)
    #print(end)
    i = 0;
    random_val = []
    while i<cols:
        #print(start[i]+" "+end[i])
        random_val.append(Rand(start[i],end[i], num))
        #print(start[i])
        #print(end[i])
        i = i+1
    return random_val
#print(random_rows)


def Rand(start, end, num): 
    res = [] 
  
    for j in range(num): 
        res.append(random.randint(start, end)) 
  
    return res
  
def getNoisePlaces(dataSet,percent):
    size = dataSet.shape
    rows = size[0]
    num = (int)(rows*(percent/100))
    start = 0
    end = rows-1
    random_rows=Rand(start, end, num)
    return random_rows

def readData():
    #read both csv files to np array
    print("Reading Data....")

    #files = os.listdir("../NumericData/")
    files = ['clean_adult.csv']
    for key in range(0, len(files)):
        orginal_dataset = pd.read_csv("../NumericDataSet/"+files[key])
        size = orginal_dataset.shape
        rows = size[0]
        cols = size[1]
        
        #orginal_dataset = orginal_dataset[:int(rows/7)]
        
        #df1 = pd.DataFrame(orginal_dataset)
        #df1.to_csv("../NumericData/clean_"+files[key])
        #print("Saved New File As ../NumericData/clean_"+files[key])
        
        print("Using "+files[key])
    
        copy_dataset=orginal_dataset[['workclass_num', 'education.num', 'marital_num', 'race_num', 'sex_num', 'rel_num', 'capital.gain', 'capital.loss']]
        other_dataset=orginal_dataset[['workclass','fnlwgt','education','marital.status','occupation','relationship','race','sex','hours.per.week','native.country','income','age','over50K']]
                
        percent = [10,20,30,40,50]
        for x in percent:
            print("Adding "+str(x)+"% Noise")
            random_data = getNoiseData(copy_dataset,x)
            
            copy_dataset = copy_dataset.reindex(['workclass_num', 'education.num', 'marital_num', 'race_num', 'sex_num', 'rel_num', 'capital.gain', 'capital.loss'], axis=1)
            other_dataset = other_dataset.reindex(['workclass','fnlwgt','education','marital.status','occupation','relationship','race','sex','hours.per.week','native.country','income','age','over50K'], axis=1)
        
            size = copy_dataset.shape
            rows = size[0]
            cols = size[1]
        
            print(size)
            k = 0
            while k<cols:
                random_places = getNoisePlaces(copy_dataset,x)
                #print(len(random_places))
                j = 0
                print(k)
                while j<len(random_places):
                    a = random_data[k][j]
                    row2change = random_places[j]
                    #print(row2change)
                    copy_dataset.iloc[row2change,k] = a
                    
                    #copy_dataset.itemset((j,k),a)
                    
                    #print(copy_dataset[j,k])
                    #print(random_data[k][j])
                    j = j+1
                k = k+1
            #our_dataset = copy_dataset.append(other_dataset, ignore_index=True)
            our_dataset = pd.concat([copy_dataset,other_dataset], axis=1)
            #df = pd.DataFrame(our_dataset)
            our_dataset.to_csv("../NumericDataSet/"+str(x)+"_noisy_"+files[key],index=False)
            print("Saved New File As ../NumericDataSet/"+str(x)+"_noisy_"+files[key])
    print("Noise Addition Completed")


readData()


Reading Data....
Using clean_adult.csv
Adding 10% Noise
(30718, 8)
0
1
2
3
4
5
6
7
Saved New File As ../NumericDataSet/10_noisy_clean_adult.csv
Adding 20% Noise
(30718, 8)
0
1
2
3
4
